In [ ]:
import zipfile
import os

zip_path = "images.zip"
extract_path = "/content/images"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Files extracted to:", extract_path)


Files extracted to: /content/images


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

labels_df = pd.read_excel("labels.xlsx")
labels_df["label"] = labels_df["label"].map({"PB": 0, "PM": 1})

image_dir = "/content/images/images.zip"

X= []
y=[]
image_size = (224, 224)

for index, row in labels_df.iterrows():
    image_path = os.path.join(image_dir, row["Image"])

    if not os.path.exists(image_path):
        print(f"Missing file: {image_path}")
        continue

    img = cv2.imread(image_path)

    if img is None:
        print(f"Corrupt or unsupported format: {image_path}")
        continue

    img = cv2.resize(img, image_size)
    img = img / 255.0

    X.append(img)
    y.append(row["label"])


X = np.array(X)
y = np.array(y)
y = to_categorical(y, num_classes=2)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=16)

model.save("breast_cancer_classifier.h5")

val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_acc:.2f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 693ms/step - accuracy: 0.5566 - loss: 0.8544 - val_accuracy: 0.7500 - val_loss: 0.6726
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.6837 - loss: 0.6645 - val_accuracy: 0.7500 - val_loss: 0.5553
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.7192 - loss: 0.5943 - val_accuracy: 0.7500 - val_loss: 0.5740
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.7561 - loss: 0.5997 - val_accuracy: 0.7500 - val_loss: 0.5938
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.7048 - loss: 0.6081 - val_accuracy: 0.7500 - val_loss: 0.5609
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.7054 - loss: 0.6406 - val_accuracy: 0.7500 - val_loss: 0.5766
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.7064 - loss: 0.6069 - val_accuracy: 0.7500 - val_loss: 0.5584
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.7215 - loss: 0.5885 - val_accuracy: 0.7500 - val_loss: 0.587

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.7500 - loss: 0.5753
Validation Accuracy: 0.75


In [ ]:
import tensorflow as tf
import numpy as np
import cv2

# Load the trained model
model = tf.keras.models.load_model("breast_cancer_classifier.h5")

# Define image size (same as during training)
image_size = (224, 224)

# Define class labels
class_labels = {0: "PB", 1: "PM"}  # Modify as per your dataset labels

def predict_image(image_path):
    # Load image
    img = cv2.imread(image_path)

    if img is None:
        print(f"Error: Could not read image {image_path}")
        return

    # Preprocess image
    img = cv2.resize(img, image_size)
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    # Make prediction
    predictions = model.predict(img)
    predicted_class = np.argmax(predictions, axis=1)[0]  # Get class with highest probability
    confidence = np.max(predictions) * 100  # Convert to percentage

    # Print result
    print(f"Prediction: {class_labels[predicted_class]} ({confidence:.2f}% confidence)")

# Example usage: Provide the image path correctly
predict_image("/content/IIR0001.jpg")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 890ms/step
✅ Prediction: PB (68.73% confidence)
